# Target Factor Model: handling data

# Bibliotecas 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import statsmodels.api as sm

# Funções

In [2]:
# CRIANDO LAGS 

def create_lags(df, lags):
    for col in df.columns:
        for lag in lags:
            df.loc[:,col+"_"+str(lag)] = df[col].shift(lag)

# Dados

In [3]:
# Lendo meta_data

meta_data = pd.read_csv('../../../data/meta-data-full.csv')
meta_data.drop(['tcode', 'transf_code'], axis = 1, inplace = True)
meta_data

,group,category,code,name_df,name,source,frequency,original,order,transf,days_lag,where
0,0,Produção,22099,gdp,PIB trimestral - Dados observados - preços de ...,IBGE,Q,Index,NaN,pctchg_12,63.0,BCB
1,0,Produção,24363,ibc_br,Índice de Atividade Econômica do Banco Central...,BCB,M,Index,NaN,diff_12,52.0,BCB
2,0,Produção,4380,mon_gdp,PIB mensal - Valores correntes (R$ milhões)\t,BCB,M,R$,NaN,diff_12,48.0,BCB
3,0,Produção,21859,ind_prod,Indicadores da produção (2012=100) - Produção ...,IBGE,M,Index,NaN,diff_12,30.0,BCB
4,0,Produção,21861,ind_prod_ext,Indicadores da produção (2012=100) - Produção ...,IBGE,M,Index,NaN,diff_12,30.0,BCB
...,...,...,...,...,...,...,...,...,...,...,...,...
190,9,Google Trends,empreendedor,gt_empreendedor,Google Trends - Empreendedor,Google,M,Trend Index,NaN,diff_diff_12,NaN,gtrendsR
191,9,Google Trends,abrir empresa,gt_empresa,Google Trends - Abrir empresa,Google,M,Trend Index,NaN,diff_12,NaN,gtrendsR
192,9,Google Trends,credito,gt_credito,Google Trends - Crédito,Google,M,Trend Index,NaN,diff_diff_12,NaN,gtrendsR
193,9,Google Trends,dolar,gt_dolar,Google Trends - Dólar,Google,M,Trend Index,NaN,diff_12,NaN,gtrendsR


In [4]:
# Carregando dados tratados

data = pd.read_csv('../../../data/data_full_tf12.csv')
# data['Date'] = pd.date_range(start = '2002-03-01', end = '2022-09-01', freq = 'QS-DEC') --> se fosse para trimestre 
data['Date'] = pd.date_range(start = '2002-01-01', end = '2022-09-01', freq = 'MS')
data.set_index('Date', inplace = True)

# Agregando as séries pela média e substituindo NAs pela mediana

qdata = (data.groupby(pd.PeriodIndex(data.index, freq = 'Q'), axis = 0)).mean()
cols = qdata.columns

qdata

,gdp,ibc_br,mon_gdp,ind_prod,ind_prod_ext,ind_prod_transf,ind_cap_goods,ind_prod_int_goods,ind_prod_man,ind_prod_dur_goods,...,gt_politica,gt_aluguel,gt_seguro,gt_governo,gt_viagem,gt_empreendedor,gt_empresa,gt_credito,gt_dolar,gt_financiamento
Date,,,,,,,,,,,,,,,,,,,,,
2002Q1,NaN,2.450000,29367.700000,0.700000,2.800000,0.800000,3.800000,0.300000,0.800000,2.300000,...,0.375000,0.750000,0.375000,0.000000,1.375000,-0.250000,-1.125000,0.000000,2.375000,1.250000
2002Q2,NaN,2.450000,29367.700000,0.700000,2.800000,0.800000,3.800000,0.300000,0.800000,2.300000,...,0.375000,0.750000,0.375000,0.000000,1.375000,-0.250000,-1.125000,0.000000,2.375000,1.250000
2002Q3,NaN,2.450000,29367.700000,0.700000,2.800000,0.800000,3.800000,0.300000,0.800000,2.300000,...,0.375000,0.750000,0.375000,0.000000,1.375000,-0.250000,-1.125000,0.000000,2.375000,1.250000
2002Q4,NaN,2.450000,29367.700000,0.700000,2.800000,0.800000,3.800000,0.300000,0.800000,2.300000,...,0.375000,0.750000,0.375000,0.000000,1.375000,-0.250000,-1.125000,0.000000,2.375000,1.250000
2003Q1,0.026665,2.450000,18315.000000,1.400000,5.733333,1.166667,-0.933333,3.700000,-1.666667,2.200000,...,0.375000,0.750000,0.375000,0.000000,1.375000,-0.250000,-1.125000,0.000000,2.375000,1.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021Q3,0.044305,5.106667,108263.266667,-1.200000,-0.433333,-1.266667,22.533333,-2.000000,-6.600000,-21.266667,...,0.083333,3.833333,1.375000,-18.208333,9.291667,4.666667,-3.875000,2.416667,3.958333,-12.958333
2021Q4,0.021351,0.723333,86861.833333,-6.033333,0.400000,-6.933333,5.166667,-4.733333,-12.433333,-28.033333,...,0.750000,3.041667,2.666667,-2.500000,10.208333,-4.500000,-1.583333,-1.333333,-2.666667,-8.166667
2022Q1,0.024284,2.423333,54362.233333,-3.233333,-1.533333,-3.433333,1.200000,-0.600000,-9.566667,-20.466667,...,2.500000,8.250000,2.541667,4.750000,14.083333,2.541667,1.166667,1.458333,35.416667,0.208333


# Variável-alvo vs. variáveis explicativas

In [5]:
X = qdata.drop('gdp', axis = 1)
y = qdata.gdp
y.fillna(y.median(), inplace = True)

In [6]:
y.loc['2003Q1':]

Date
2003Q1    0.026665
2003Q2    0.007798
2003Q3    0.006276
2003Q4    0.005871
2004Q1    0.038739
            ...   
2021Q3    0.044305
2021Q4    0.021351
2022Q1    0.024284
2022Q2    0.036869
2022Q3    0.035828
Freq: Q-DEC, Name: gdp, Length: 79, dtype: float64

# Pré-seleção de variáveis

In [7]:
X_new = X[['ibc_br']]
lags = [1, 2, 3, 4]
create_lags(X_new, lags)
X_new['gdp_1'] = y.shift(1)
X_new

C:\Users\Henri\AppData\Local\Temp\ipykernel_4580\1043497302.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,col+"_"+str(lag)] = df[col].shift(lag)
C:\Users\Henri\AppData\Local\Temp\ipykernel_4580\1043497302.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,col+"_"+str(lag)] = df[col].shift(lag)
C:\Users\Henri\AppData\Local\Temp\ipykernel_4580\1043497302.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,ibc_br,ibc_br_1,ibc_br_2,ibc_br_3,ibc_br_4,gdp_1
Date,,,,,,
2002Q1,2.450000,NaN,NaN,NaN,NaN,NaN
2002Q2,2.450000,2.450000,NaN,NaN,NaN,0.022836
2002Q3,2.450000,2.450000,2.450000,NaN,NaN,0.022836
2002Q4,2.450000,2.450000,2.450000,2.450000,NaN,0.022836
2003Q1,2.450000,2.450000,2.450000,2.450000,2.450000,0.022836
...,...,...,...,...,...,...
2021Q3,5.106667,17.100000,1.900000,-0.680000,-4.973333,0.124139
2021Q4,0.723333,5.106667,17.100000,1.900000,-0.680000,0.044305
2022Q1,2.423333,0.723333,5.106667,17.100000,1.900000,0.021351


In [8]:
X_new.loc['2003Q1':].isna().sum()

ibc_br      0
ibc_br_1    0
ibc_br_2    0
ibc_br_3    0
ibc_br_4    0
gdp_1       0
dtype: int64

In [9]:
# Loop para computar os p-valores e selecionar as variáveis mais relevantes

columns = X.columns
pvals = []
variables = []

for col in columns:
    X_new = X[[col]].copy()
    lags = [1, 2, 3, 4]
    create_lags(X_new, lags)
    X_new['gdp_1'] = y.shift(1)
    X_reg = X_new.loc['2003Q2':].copy()
    y_reg = y.loc['2003Q2':].copy()
    X_reg = sm.tools.add_constant(X_reg, prepend = True)
    model = sm.OLS(y_reg, X_reg).fit()
    pvals.append(model.pvalues[col])
    variables.append(col)

print(variables, pvals)

['ibc_br', 'mon_gdp', 'ind_prod', 'ind_prod_ext', 'ind_prod_transf', 'ind_cap_goods', 'ind_prod_int_goods', 'ind_prod_man', 'ind_prod_dur_goods', 'ind_prod_nondur_goods', 'ind_prod_cc_materials', 'ind_real_sales', 'vehi_prod', 'pass_cars_prod', 'truck_prod', 'bus_prod', 'steel_prod', 'road_flow_gross_vehi', 'ipca', 'ipca_ex', 'ipca_exi', 'ipca_exii', 'ipca_exiii', 'inpc', 'igpm', 'igpdi', 'incc_m', 'incc_di_mo', 'incc_di_servmat', 'ipatac_m', 'ipc_br', 'ipa', 'ipc_fipe', 'commdt_index_br', 'commdt_index_agro', 'commdt_index_metal', 'sinapi', 'ccb_sp', 'ccb_bh', 'ccb_rj', 'ccb_bsb', 'ccb_sal', 'wage_mass_ind', 'new_caged', 'caged_agro', 'caged_ind_ext', 'caged_ind_transf', 'caged_const', 'caged_com', 'caged_servi', 'hrs_wkd_ind_transf', 'unem_rate_pnad', 'total_sales', 'const_sales', 'com_sales', 'forn_app_sales', 'fuel_sales', 'nondur_congoods_sales', 'vest_sales', 'vehi_sales', 'medcare_sales', 'books_sales', 'offcomm_sales', 'others_sales', 'vehi_total_sales_fenabrave', 'pass_vehi_sa

In [10]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    gdp   R-squared:                       0.626
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     19.85
Date:                Mon, 19 Jun 2023   Prob (F-statistic):           1.82e-13
Time:                        14:55:03   Log-Likelihood:                 186.85
No. Observations:                  78   AIC:                            -359.7
Df Residuals:                      71   BIC:                            -343.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0050      0

In [11]:
target_df = pd.DataFrame({'variables':variables,
                         'pvals':pvals})
target_df['pvals'] = target_df['pvals'].round(5)
target_df.sort_values(by = ['pvals'], ascending = True)

,variables,pvals
0,ibc_br,0.00000
113,serv_curr_situ_index_fgv,0.00000
55,forn_app_sales,0.00000
54,com_sales,0.00000
53,const_sales,0.00000
...,...,...
92,net_debt_per_gdp_pe,0.91221
157,gt_jobs,0.92393
86,idp_gdp_twelve_mon,0.92624
175,gt_renda,0.99658


In [12]:
selected = target_df.query('pvals <= 0.01')
dropped = target_df.query('pvals > 0.01')
dropped

,variables,pvals
18,ipca,0.14199
20,ipca_exi,0.04790
21,ipca_exii,0.02805
23,inpc,0.29767
24,igpm,0.01861
...,...,...
186,gt_seguro,0.13871
187,gt_governo,0.52500
189,gt_empreendedor,0.31785
190,gt_empresa,0.31306


# Tirando as variáveis não selecionadas e salvando o dataframe final

In [13]:
drop_list = list(dropped['variables'].values)
target_data = data.drop(drop_list, axis = 1).copy()
target_data.to_csv('../../../data/data_target_full_tf12.csv')
# target_data.to_excel('../../../data/data_target_gtcat_tf12.xlsx')
target_data

,gdp,ibc_br,mon_gdp,ind_prod,ind_prod_ext,ind_prod_transf,ind_cap_goods,ind_prod_int_goods,ind_prod_man,ind_prod_dur_goods,...,gt_sports,gt_crise,gt_desemprego,gt_segdesemp,gt_economia,gt_emprestimo,gt_aluguel,gt_viagem,gt_dolar,gt_financiamento
Date,,,,,,,,,,,,,,,,,,,,,
2002-01-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,2.000,0.625,-0.500,0.750,-1.625,0.125,0.750,1.375,2.375,1.250
2002-02-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,2.000,0.625,-0.500,0.750,-1.625,0.125,0.750,1.375,2.375,1.250
2002-03-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,2.000,0.625,-0.500,0.750,-1.625,0.125,0.750,1.375,2.375,1.250
2002-04-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,2.000,0.625,-0.500,0.750,-1.625,0.125,0.750,1.375,2.375,1.250
2002-05-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,...,2.000,0.625,-0.500,0.750,-1.625,0.125,0.750,1.375,2.375,1.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,NaN,5.07,101660.0,-0.3,-10.4,1.3,6.4,-2.3,2.2,-3.4,...,7.250,3.125,-1.875,-2.125,-0.375,-6.000,10.625,11.125,26.625,-3.875
2022-06-01,0.036869,4.44,102487.4,-0.6,-5.9,0.2,-0.4,-3.0,3.8,2.0,...,9.625,1.000,2.000,0.375,-1.875,-0.250,7.500,8.500,-2.375,-9.750
2022-07-01,NaN,6.44,99509.7,-0.4,-4.3,0.2,-6.6,-1.8,3.7,-0.8,...,6.500,1.875,2.125,1.625,-0.625,14.250,7.625,12.500,7.875,-7.375


In [14]:
final_cols = list(target_data.columns)
final_cols

['gdp',
 'ibc_br',
 'mon_gdp',
 'ind_prod',
 'ind_prod_ext',
 'ind_prod_transf',
 'ind_cap_goods',
 'ind_prod_int_goods',
 'ind_prod_man',
 'ind_prod_dur_goods',
 'ind_prod_nondur_goods',
 'ind_prod_cc_materials',
 'ind_real_sales',
 'vehi_prod',
 'pass_cars_prod',
 'truck_prod',
 'bus_prod',
 'steel_prod',
 'road_flow_gross_vehi',
 'ipca_ex',
 'ipca_exiii',
 'incc_m',
 'incc_di_mo',
 'wage_mass_ind',
 'new_caged',
 'caged_agro',
 'caged_ind_ext',
 'caged_ind_transf',
 'caged_const',
 'caged_com',
 'caged_servi',
 'hrs_wkd_ind_transf',
 'unem_rate_pnad',
 'total_sales',
 'const_sales',
 'com_sales',
 'forn_app_sales',
 'fuel_sales',
 'vest_sales',
 'vehi_sales',
 'books_sales',
 'offcomm_sales',
 'others_sales',
 'vehi_total_sales_fenabrave',
 'pass_vehi_sales',
 'light_comm_vehi_sales',
 'truck_sales',
 'bus_sales',
 'vehi_total_sales_anfavea',
 'vehi_domestic_sales',
 'energy_comm',
 'energy_ind',
 'energy_total',
 'co_gas',
 'co_oilderiv_total',
 'co_alc_total',
 'ide_mon',
 'pr_cen

# Verificando os grupos de variáveis

In [18]:
series = list(target_data.columns)
n_series = np.arange(0, (target_data.shape[1]))
series_df = pd.DataFrame(n_series, index = series)

In [20]:
# Criando df para mapear grupo no dataframe de contribuição dos componentes

map_comp = meta_data[['category', 'name_df']].copy()
map_comp.set_index('name_df', inplace = True)
map_comp = map_comp.iloc[1:]

robust_data = pd.merge(series_df, map_comp, left_index=True, right_index=True)
robust_data['category'].value_counts()

Produção               19
Vendas                 17
Google Trends          16
Expectativas           15
Mercado de Trabalho    10
Setor Externo           7
Financeiras             7
Energia                 6
Preços                  4
Setor Público           2
Name: category, dtype: int64